In [ ]:
# run this cell - obligatory imports in every file
from datascience import *
import pandas as pd
from pandas import read_stata
import numpy as np

# A note - throughout the code there are occasionally commands that are not run becauase they 
# are proceeded by a # sign - but they are left there in case someone wants to remove the # sign
# and use the code.  

## Baseline Survey 
This is our first look at a survey dataset.  These are a limited set of questions/answers from a simple and short baseline survey. However it is a lot bigger and messier than the datasets we have been seeing in Data8. Data variable names follow along with the survey below, referred to by the section, a,b,c... number, 1,2,3... and a few words about the question. The purpose of this lab will be to get a familiarity with the dataset, to look at some background descriptor variables of the households, to start to think about missing values and coding of subsets of the data.  The main framework of this Lab is checking the randomization by seeeing if the different arms of the study are balanced across some of the key baseline variables.  

## The surveys that illustrate the raw data names are in a Box folder linked here
## You have to go and look through this survey to understand the variables
https://goo.gl/TzzvLb

##  A note on Pandas vs datascience/tables
- The main data8 class is taught mostly using a Python package designed specifically for the class called "datascience"
- A very popular package for statistical analysis is called "pandas". 
- In this Lab, for a few reasons, I have written commands mostly in Pandas


In [ ]:
# Make it so Pandas can display many columns
# there are 200 something columns in this dataset
pd.set_option('display.max_columns', 500)

In [ ]:
# read in stata data set using pandas ( pd) 
#pd.read_stata('stata.dta') is the general command
#pd.read_stata('bwm_child_EVDvars.dta') is the specific file we need to have in our folder
WGP_baseline = pd.DataFrame(pd.read_stata('WGP_baseline_Data8.dta'))
# this creates the pandas dataframe that we will refer to when we run additional commands


WGP_baseline


## Misssing values ~ NaN
if you look through the dataset above, and scroll to the right a ways to some of the last variables, you will notice that that there are a lot of cells with NaN, which means a missing value. For these cells no data was entered at the time of data entry. In some cases it may be appropriate to enter a zero and carry on with the analyis.  



In [ ]:
# There are a lot of missing values in the data, so we can make a copy of the dataset/dataframe
# that has zeros in the place of 'nan' - mising data values
WGP_base_dfna=WGP_baseline.fillna(0)
# this is a second dataframe that we can call with a different name
WGP_base_dfna.head(10)

In [ ]:
# Lets make a datascience table at the same time
WG_base_table= Table.from_df(WGP_baseline)
# this is a table that we can use with the data8/datascience commands
#WG_base_table

In [ ]:
# Here is a list of all of the possible categories / columns
list(WGP_baseline)
# Look at the variable names, and then look at the survey form to find the concordance of codes


### Front Page information - A variables
 - household id, 
 - spring id, 
 - interviewer id


### Information about respondent - B variables 
- tribe
 - education
 - age
 - gender 
 - group membership
 
 
### Water Guard Use - C variables
for Waterguard (WG),(survey questions on other chlorine not in this dataset) 

- c1a - have you ever heard of WG
- c2a - have you ever used WG
- c3a - is your water currently treated with WG
- c4a - have you used WG in past month



### Durable / Capital Goods - D variables
 - electricity  / latrine / iron roof  ( yes / no) 
 - bicycle/ radio / hoe / beds ( number of items owned)
 - number of animals (number)
 
 
### Child Health - E variables
 -  number of kids under 5 =  e1_num_kids_under_5
 
e2_ This table becomes tricky because it has a different format 
Each kid in the table is numbered 01, 02 etc
and then the subsequent questions keyed to that child number
e2e_01_d_diarrhea, e2e_02_d_diarrhea, e2e_03_d_diarrhea
for four diseases:

 - cough
 - Diarrhea
 - fever_malaria
 - vomiting
 


 

In [ ]:
# There is an additional variable called treatment arm
# Arm 1 is control, and Arms 2-7 are different types of treatment interventions
# I will elaborate on these arms more in class, but for now
# Arm 1 - Control
# Arm 2 - Household Script
# Arm 3 - Community Script
# Arm 4 - HH + Community Script
# Arm 5 - Flat-Fee Promoter + Coupons
# Arm 6 - Incentivized Promoter + Coupons
# Arm 7 - Incentivized Promoter + Dispenser at Spring

#Pandas
WGP_baseline.groupby("treatment_arm").size()
# This should show us how many households are in each treatment arm

#WGP_base_dfna.groupby("treatment_arm").size()

#Datascience
#WG_base_table.group("treatment_arm")

In [ ]:
# Lets start with on of the b variables, b5_tribe
#Pandas
WGP_baseline.groupby("b5_tribe").size()
#WGP_base_dfna.groupby("b5_tribe").size()
#This should show us how many answers for each of the tribe codes.

#Datascience
#WG_base_table.group("b5_tribe")

In [ ]:
#  But these labels arent super useful, they refer to codes on a codesheet
# The code sheet is at https://goo.gl/S5CiDJ
# this is how to relabel in Pandas
WGP_baseline["Tribe"] = WGP_baseline["b5_tribe"].astype("category")
WGP_baseline["Tribe"].cat.categories = ["Luhya – Samia", "Luhya – Nyala", "Luhya – Khayo", "Luhya – Marachi", "Luhya – Other", "Luo", "Teso", "Other"]
WGP_baseline.groupby("Tribe").size()




In [ ]:
# Crosstab is a frequency count by category in Pandas
pd.crosstab(index=WGP_baseline["Tribe"], columns="count")

In [ ]:
#  Try making a frequency percentage from a oneway crosstab table
tribe_tab = pd.crosstab(index=WGP_baseline["Tribe"],columns="count") 
print  ( tribe_tab.sum(), "\n")  # Sum the counts
print (tribe_tab.shape, "\n")   # Check number of rows and cols
tribe_tab/tribe_tab.sum()

In [ ]:
# This is a command to display only 3 decimal places - useful for percents in next section
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# Next we will move to a balance check - 
#checking the number in each tribe, looking across the different treatment arms
# Simple numberical cross tab of tribe vs treatment arm
# This will yield counts of who is in which bin
pd.crosstab(WGP_baseline.Tribe,WGP_baseline.treatment_arm, margins=True)

In [ ]:
#This table allows us to check if the tribe composition is balanced across arms 
#However it is hard to compare because there are different total numbers of HHs in each arm
# Percentages would be better
# Table of tribe  vs treatment arm but with Percentages
tribeVStreatment = pd.crosstab(index=WGP_baseline["Tribe"], 
                            columns=WGP_baseline["treatment_arm"],
                             margins=True)   # Include row and column totals
tribeVStreatment/tribeVStreatment.loc["All"]


## Question 1

- 1.1 Explain the previous table clearly and concisely, as if you were explaining it to someone who didnt know the back story
- What are the rows, what are the columns, why are we doing this?
- 1.2 This is a balance check, where we can check whether each treatment arm has a roughly similar composition.  How does the randomization look?
- 1.3 What if you wanted to statistically test whether the arms were balanced? Can you suggest how to do this?

In [ ]:
### Moving on  to  WaterGuard Use
WGP_baseline.groupby("c2a_wg_used_ever").size()

In [ ]:
# FWIW Pandas also has a describe command
# We havent covered these stats in Data8 yet but they are mostly
WGP_baseline['c2a_wg_used_ever'].groupby(WGP_baseline['treatment_arm']).describe()

In [ ]:
#  Make a new column/variable with 1/2 answers translated into Yes/NO
WGP_baseline["WG Ever Use"] = WGP_baseline["c2a_wg_used_ever"].astype("category")
WGP_baseline["WG Ever Use"].cat.categories = ["Yes", "No" ]
WGP_baseline.groupby("WG Ever Use").size()

In [ ]:
# And the balance check for Water Guard use across Arms 
#Table of treatment arm vs Water Guard Use but with Percentages
WGUsec2VStreatment = pd.crosstab(index=WGP_baseline["WG Ever Use"], 
                            columns=WGP_baseline["treatment_arm"],
                             margins=True)   # Include row and column totals
WGUsec2VStreatment/WGUsec2VStreatment.loc["All"]



##  Question 2
- 2.1 Explain the previous table clearly and concisely, as if you were explaining it to someone who didnt know the back story
- What are the rows, what are the columns, why are we doing this?
- 2.2 Balance Check - How does the randomization look?
- 2.3 Why is this table more important than in Question 1?

In [ ]:
# Now lets check and see how many households are currently using WG -
# Variable name is c3a_wg_water_currently_treat

WGP_baseline.groupby("c3a_wg_water_currently_treat").size()

In [ ]:
# Do you notice a problem here? Look at the total numbers reported in the output above

# We can do the same percentage tables for the balance check 
# but maybe there's a problem?
# Table of treatment arm vs Water Guard Use but with Percentages
WG_c3VStrt = pd.crosstab(index=WGP_baseline["c3a_wg_water_currently_treat"], 
                            columns=WGP_baseline["treatment_arm"],
                             margins=True)   # Include row and column totals
WG_c3VStrt/WG_c3VStrt.loc["All"]
# In this case the 1's represent the percent  answering yes, out of all those who answered the question


In [ ]:
#Earlier we created a dataset where the missing values "nan" were replaced with zeros
# If we use this dataset we would have percents over the total population surveyed

#Table of treatment arm vs Water Guard Use,  Percentages, with missing values substituted with zeros

WG_c3VStrt = pd.crosstab(index=WGP_base_dfna["c3a_wg_water_currently_treat"], 
                            columns=WGP_base_dfna["treatment_arm"],
                             margins=True)   # Include row and column totals
WG_c3VStrt/WG_c3VStrt.loc["All"]
# In this case the 1's represent the percent  answering yes, out of all those who were surveyed


## Question 3
 - 3.1 Discuss the problem with this c3a variable
 - Discuss why the first table is not representative of the whole sample
 - 3.2 Compare the percent using WG ( answering 1) in the first table to the second table 
 - 3.3 What can we conclude about the average household's Waterguard use from the second table? 
 

In [ ]:

# Lets look across the whole sample at the rest of the chlorine variables
# Look at any other of the Chlorine variables?
#' c1a_wg_heard_ever',
#  'c2a_wg_used_ever',
# ' c3a_wg_water_currently_treat',
#  'c4a_wg_used_past_month',
#  'c5a_wg_received_past_month',



In [ ]:
# looking at Household durable goods
# For example whether the household has an iron roof
WGP_baseline.groupby("d11_iron_roof").size()



In [ ]:
# Table of treatment arm vs Iron Roof   with Percentages
IronRoofVStreatment = pd.crosstab(index=WGP_baseline["d11_iron_roof"], 
                            columns=WGP_baseline["treatment_arm"],
                             margins=True)   # Include row and column totals
IronRoofVStreatment/IronRoofVStreatment.loc["All"]




## Question 4.1
 - What would iron roof ( vs thatch roof) be an indicator of? 
 - Write 2-3 sentences discussing the previous table
 
## Question 4.2
 - Generate the same table for latrines, and write 2-3 sentences to discuss
 ( insert a new code cell and markdown cell below)
 
## Question 4.3
 - Explore 3 of the other d variables, to describe the sample population as a whole, ( and not by treatment arm) 
( insert cells as needed ) 



In [ ]:
# Child variables e1 and e2
WGP_baseline.groupby("e1_num_kids_under_5").size()

# or should we use the dataframe with NaN missing values replaced by zeros?
#WGP_base_dfna.groupby("e1_num_kids_under_5").size()

In [ ]:
# Look at the child table in the survey
# The child table has its own dimension, one entry for each child in the household under 5

# For now lets just look at the first child in the survey, who is number 01.
#
#  for Child Diarrhea  - varname for first child only is e2e_01_d_diarrhea

WGP_baseline.groupby("e2e_01_d_diarrhea").size()

In [ ]:
# Table of treatment arm vs Child Diarrhea in kid 1    with Percentages
Child1DiarrheaVStreatment = pd.crosstab(index=WGP_baseline["e2e_01_d_diarrhea"], 
                            columns=WGP_baseline["treatment_arm"],
                             margins=True)   # Include row and column totals
Child1DiarrheaVStreatment/Child1DiarrheaVStreatment.loc["All"]




### Question 5
- 5.1 Please look up the definition of diarrhea in the survey, what is the specific definition of diarrrhea and what is the time period

- 5.2 Of the four diseases tracked what is the most common, what is the least common across the whole sample (again for the first child only)

- 5.3 Only looking at the first child is not using all the information. We would want to make a composite variable across all children in the household.  Something like total number of children with diarrhea / total children in household.  And from there to boys with diarrhea, girls with diarrhea, children with multiple diseases, children with single diseases etc.  Please discuss a strategy to program this. Provide code examples if you can, or outline a coding approach with words.  